In [1]:
delete from [Theme_Park].[Maintenance_Calendar]
go
drop table [Theme_Park].[Maintenance_Notification]
go
create table [Theme_Park].[Maintenance_Notification]
(
    notification_id int identity(1,1),
    maintenance_id int not null,
    completion datetime not null,
    billed_hours int,
    amount smallmoney,
)

: Msg 3701, Level 11, State 5, Line 1
Cannot drop the table 'Theme_Park.Maintenance_Notification', because it does not exist or you do not have permission.

Commands completed successfully.

Total execution time: 00:00:00.121

## Trigger TRIGGER_MANAGER_NOTIFICATION_ON_MAINTENANCE_COMPLETION:

This trigger is intended to create notification entries whenever a maintenance event is updated or inserted
with a completion time set. These notifications have 


In [85]:
update [Theme_Park].[Department] set supervisor_ssn=(select ssn from [Theme_Park].[Employee] where fname='Fernando') where dept_id=1
go
update [Theme_Park].[Department] set supervisor_ssn=(select ssn from [Theme_Park].[Employee] where fname='Carissa') where dept_id=2
go
select fname, lname, dept_name from [Theme_Park].[Employee] as E, [Theme_Park].[Department] as D where D.supervisor_ssn=E.ssn

(1 row affected)

(1 row affected)

(2 rows affected)

Total execution time: 00:00:00.163

fname,lname,dept_name
Fernando,Recinos,Rides
Carissa,Becerra,Food Court


In [1]:
drop trigger [Theme_Park].[Trigger_Manager_Notification_On_Maintenance_Completion]
go
create trigger [Theme_Park].[Trigger_Manager_Notification_On_Maintenance_Completion]
on [Theme_Park].[Maintenance_Calendar]
after insert,update
as
BEGIN
    set nocount on
    declare 
        @id int,
        @Completion_Date datetime,
        @Hours int,
        @Amount smallmoney,
        @Description varchar(50)
    declare cur cursor read_only for 
        select Maint_ID, Maint_Completion, Billed_Hours, Invoice_amount, Maint_Description from inserted
    open cur
    fetch next from cur into @id, @Completion_Date, @Hours, @Amount, @Description
    while @@fetch_status = 0
    begin
        if (@Completion_Date is not null)
        begin
            insert into [Theme_Park].[Maintenance_Notification]
            values (@id, @Completion_Date, @Hours, @Amount, @Description)
        end
        fetch next from cur into @id, @Completion_Date, @Hours, @Amount, @Description
    end
    close cur
    deallocate cur
end
    

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.112

## Trigger TRIGGER\_MANAGER\_NOTIFICATION\_ON\_EMPLOYEE\_CHANGE:

This trigger fires whenever an employee undergoes certain kinds of changes. Two different kinds of notifications are created in the \[Theme\_Park\].\[Manager\_Notifications\] table:

* If the employee's payrate or salary change, a notification is directed towards the employee's supervisor and the supervisor of their department.
* If the employee's department or role change, a notification is directed towards the employee's new supervisor and the supervisor of their new department.

This also fires when employees are added, letting managers know about the payscale and responsibilities of new hires

In [88]:
drop trigger [Theme_Park].[Trigger_Manager_Notification_On_Employee_Change]
go
create trigger [Theme_Park].[Trigger_Manager_Notification_On_Employee_Change]
on [Theme_Park].[Employee]
after insert,update
as
BEGIN
    set nocount on
    declare 
        @fname varchar(20),
        @lname varchar(20),
        @ssn numeric(9,0),
        @role varchar(20),
        @dept_id int,
        @payrate money,
        @supervisor_ssn int,
        @salaried bit
    -- Here we use COLUMNS_UPDATED to check for the updated columns of the rows
    -- This goes from right to left, as a bitmask. A 1 in the nth-most bit from the right, means that column n of [Theme_Park].[Employee] was updated
    -- Payrate is column 12, salaried is column 11, department is column 8, role is column 9
    -- Therefore payrate is on the 4th bit of the second byte, salaried on the 3rd of the second byte, supervisor ssn on the 2nd of the second byte, 
    -- role on the 1st bit of the second byte, and department on the 8th bit of the first byte
    declare @payrate_bitmask int = 8,
            @salaried_bitmask int = 4,
            @department_bitmask int = 128,
            @role_bitmask int = 1,
            @super_bitmask int = 2
    declare cur cursor read_only for 
        select fname, lname, ssn, dept_id, role, payrate, salaried, supervisor_ssn from inserted
    open cur
    fetch next from cur into @fname, @lname, @ssn, @dept_id, @role, @payrate, @salaried, @supervisor_ssn
    while @@fetch_status = 0
    begin
        declare @updated_columns varbinary(2) = COLUMNS_UPDATED()
        declare @first_byte varbinary = SUBSTRING(@updated_columns, 1, 1)
        declare @second_byte varbinary = SUBSTRING(@updated_columns, 2, 1)
        declare @dept_head_ssn int = (select supervisor_ssn from [Theme_Park].[Department] where dept_id = @dept_id)
        -- If the employee hasn't been assigned a department or their department has no supervisor, it doesn't make sense to make notifications here, so we just give up
        if (@dept_head_ssn is null or @supervisor_ssn is null) 
        begin
            fetch next from cur into @fname, @lname, @ssn, @dept_id, @role, @payrate, @salaried, @supervisor_ssn
            continue
        end
        declare @super_fname varchar(20) = (select fname from [Theme_Park].[Employee] where ssn = @supervisor_ssn),
                @super_lname varchar(20) = (select lname from [Theme_Park].[Employee] where ssn = @supervisor_ssn)
        if ((@second_byte & @payrate_bitmask <> 0) or (@second_byte & @salaried_bitmask <> 0))
        begin
            insert into [Theme_Park].[Manager_Notifications]
            values 
            (@supervisor_ssn, @fname, @lname, @dept_id, @role, @payrate, @salaried, @super_fname, @super_lname, 'P'),
            (@dept_head_ssn, @fname, @lname, @dept_id, @role, @payrate, @salaried, @super_fname, @super_lname, 'P')
        end
        if ((@first_byte & @department_bitmask <> 0) or (@second_byte & @role_bitmask <> 0) or (@second_byte & @super_bitmask <> 0))
        begin
            insert into [Theme_Park].[Manager_Notifications]
            values 
            (@supervisor_ssn, @fname, @lname, @dept_id, @role, @payrate, @salaried, @super_fname, @super_lname, 'R'),
            (@dept_head_ssn, @fname, @lname, @dept_id, @role, @payrate, @salaried, @super_fname, @super_lname, 'R')        
        end
        fetch next from cur into @fname, @lname, @ssn, @dept_id, @role, @payrate, @salaried, @supervisor_ssn
    end
    close cur
    deallocate cur
end
    

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.117

### Test of trigger TRIGGER\_MANAGER\_NOTIFICATION\_ON\_EMPLOYEE\_CHANGE:

We'll create a fake employee, which fires the trigger to his stated supervisor (INSERT behavior). Then we alter his department (UPDATE behavior), and then his payrate.

In [94]:
delete from [Theme_Park].[Employee] where ssn=555115555
go
delete from [Theme_Park].[Manager_Notifications]
go
insert into [Theme_Park].[Employee]
(fname, lname, ssn, gender, address, phone, date_joined, dept_id, role, supervisor_ssn, salaried, payrate, vacation_days)
values
(
    'Mario',
    'Mario',
    555115555,
    'M',
    '101 Toadstool Lane',
    7137771111,
    GETDATE(),
    (select dept_id from [Theme_Park].[Department] where dept_name = 'Rides'),
    'Plumber',
    123456788,
    0,
    7.99,
    0.0
)
go
update [Theme_Park].[Employee] set dept_id = (select dept_id from [Theme_Park].[Department] where dept_name = 'Food Court') where ssn=555115555
go
update [Theme_Park].[Employee] set payrate=30000.00, salaried=1 where ssn=555115555
go
select * from [Theme_Park].[Manager_Notifications]

(1 row affected)

(8 rows affected)

(1 row affected)

(1 row affected)

(1 row affected)

(8 rows affected)

Total execution time: 00:00:00.357

manager_ssn,employee_fname,employee_lname,dept_id,role,payrate,salaried,super_fname,super_lname,type
123456788,Mario,Mario,1,Plumber,7.99,0,Fernando,Recinos,P
123456788,Mario,Mario,1,Plumber,7.99,0,Fernando,Recinos,P
123456788,Mario,Mario,1,Plumber,7.99,0,Fernando,Recinos,R
123456788,Mario,Mario,1,Plumber,7.99,0,Fernando,Recinos,R
123456788,Mario,Mario,2,Plumber,7.99,0,Fernando,Recinos,R
838838383,Mario,Mario,2,Plumber,7.99,0,Fernando,Recinos,R
123456788,Mario,Mario,2,Plumber,30000.00,1,Fernando,Recinos,P
838838383,Mario,Mario,2,Plumber,30000.00,1,Fernando,Recinos,P


In [39]:
drop table [Theme_Park].[Manager_Notifications]
go
create table [Theme_Park].[Manager_Notifications]
(
    manager_ssn int not null,
    employee_fname varchar(20),
    employee_lname varchar(20),
    dept_id int,
    role varchar(20),
    payrate money,
    salaried bit,
    super_fname varchar(20),
    super_lname varchar(20),
    type char not null
)

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.130

In [47]:
drop proc [Theme_Park].[Proc_Get_Manager_Notifications]
go
create proc [Theme_Park].[Proc_Get_Manager_Notifications]
    @manager_ssn int
as
    select * from [Theme_Park].[Manager_Notifications] where manager_ssn=@manager_ssn order by employee_lname asc
go

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.110

In [80]:
select * from [Theme_Park].[Ticket_Reservation]
go
select * from [Theme_Park].[Maintenance_Calendar]
go
select * from [Theme_Park].[Attractions]
GO
select * from [Theme_Park].[Vendor]
go
select * from [Theme_Park].[Department]
go
select * from [Theme_Park].[Ticket]
go
select * from [Theme_Park].[Merchandise_Sale]
go
select * from [Theme_Park].[Merchandise]
go
select * from [Theme_Park].[Employee]
go

(0 rows affected)

(5 rows affected)

(3 rows affected)

(2 rows affected)

(2 rows affected)

...

dept_id,dept_name,supervisor_ssn
1,Rides,123456788
2,Food Court,NULL


Ticket_ID,Date,Ticket_Class,Price,Reservation
10,2021-09-09 00:00:00.000,Normal,11.30,NULL
11,2021-06-12 00:00:00.000,Normal,11.30,NULL
12,2021-06-14 00:00:00.000,Poor,11.30,NULL
13,2021-07-15 00:00:00.000,Normal,11.30,NULL
14,2021-09-21 00:00:00.000,Poor,11.30,NULL
15,2021-07-17 00:00:00.000,Poor,11.30,NULL
16,2021-07-16 00:00:00.000,Normal,11.30,NULL
17,2021-07-26 00:00:00.000,Premium,11.30,NULL
18,2021-07-03 00:00:00.000,Poor,11.30,NULL
19,2021-05-13 00:00:00.000,Poor,11.30,NULL


transaction_id,item_id,quantity,sale_price,attraction_id,transaction_date


item_id,name,price,merch_type
1,PinwheelHat,20.00,Clothing


fname,lname,ssn,gender,address,phone,date_joined,dept_id,role,supervisor_ssn,salaried,payrate,vacation_days
Joe,man,123456543,M,345 Main St.,8324557878,2022-11-24,1,Tech,123456789,0,20.00,12.00
Fernando,Recinos,123456788,M,90 place place Houston Tx 77004,5125743308,2022-11-10,1,tech,123456789,0,939333.00,12.00
Jerry,Boss,123456789,M,80 cougar drive,5125125125,2022-09-09,1,tech,NULL,1,2000.00,12.00
Mario,Mario,555115555,M,101 Toadstool Lane,7137771111,2022-11-11,1,Plumber,123456788,0,9.00,0.00
Carissa,Becerra,838838383,M,"12 sugarland pearland, Houston TX 78787",5126968585,2022-11-10,1,Maid,123456789,0,10000000.00,13.00
